In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

In [7]:
# Get Stock Market Data: EEM (emerging markets ETF)
import yfinance as yf
stock_info = yf.download("EEM", start="1993-02-01", end="2021-02-20")
stock_info.index = pd.to_datetime(stock_info.index)
stock_info

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-04-14,11.144444,11.222222,11.144444,11.222222,8.132662,93600
2003-04-15,11.291111,11.388889,11.291111,11.355556,8.229289,421200
2003-04-16,11.477778,11.533333,11.477778,11.486667,8.324305,9000
2003-04-17,11.550000,11.572222,11.530000,11.568889,8.383890,17100
2003-04-21,11.582222,11.583333,11.555556,11.564444,8.380672,72900
2003-04-22,11.563333,11.603333,11.527778,11.603333,8.408853,19800
2003-04-23,11.611111,11.657778,11.572222,11.657778,8.448309,31500
2003-04-24,11.566667,11.566667,11.400000,11.427778,8.281630,90000
2003-04-25,11.267778,11.267778,11.222222,11.238889,8.144743,102600


In [9]:
df1 = df[df['Year']> "2003-04-04"]

In [50]:
# stock market volatibility within 4 days after the Fed meetings, window = 4 performs the best
import numpy as np
price_changes = []
price_change_percentages = []
labels = []
window = 3
# workaround for rolling widow
for index, row in df1.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = stock_closing_sum - start_price
    price_change_percent = (stock_closing_sum - start_price) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    

df1['price_change'] = price_changes
df1['price_change_percent'] = price_change_percentages

#label =1 means high volatility, label = 1 means low volatility. High volatility is identified if VIX price change percent is far from its median (greater than 75% or smaller than 25%)
df1['label'] = df1['price_change_percent'].apply(lambda x: 1 if x> df1["price_change_percent"].quantile(0.75) or x < df1["price_change_percent"].quantile(0.25) else 0)

C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wanyi\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [51]:
X = df1
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df1['label'])

In [52]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)

In [53]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [54]:
predictions = model.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.49      0.71      0.58        24
           1       0.63      0.40      0.49        30

    accuracy                           0.54        54
   macro avg       0.56      0.55      0.53        54
weighted avg       0.57      0.54      0.53        54



In [55]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [56]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)

0.5370370370370371
              precision    recall  f1-score   support

           0       0.49      0.71      0.58        24
           1       0.63      0.40      0.49        30

    accuracy                           0.54        54
   macro avg       0.56      0.55      0.53        54
weighted avg       0.57      0.54      0.53        54

